In [46]:
# def a function evaluate the fitting goodness of dataset by Gpr model and LOOCV
import math
import warnings
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import LeaveOneOut
warnings.filterwarnings('ignore')

def eval_dataset(dataset):
    dataset = np.array(dataset)
    KErnel = 1*RBF(length_scale_bounds = (1e-2, 1e2))
    X = dataset[:, :-1]
    Y = dataset[:, -1]
    loo = LeaveOneOut()
    y_pre_set = []
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, _ = Y[train_index], Y[test_index]
        Gpr_i = GaussianProcessRegressor(kernel=KErnel,
                                        normalize_y=True,
                                        random_state=0).fit(X_train, y_train)

        y_pre = Gpr_i.predict(X_test, return_std=True)[0]
        y_pre_set.append(y_pre)
    Rvalue = PearsonR(y_pre_set, Y)
    return Rvalue


def PearsonR(X, Y):
    X = np.array(X)
    Y = np.array(Y)
    xBar = np.mean(X)
    yBar = np.mean(Y)
    SSR = 0
    varX = 0
    varY = 0
    for i in range(0, len(X)):
        diffXXBar = X[i] - xBar
        diffYYBar = Y[i] - yBar
        SSR += (diffXXBar * diffYYBar)
        varX += diffXXBar ** 2
        varY += diffYYBar ** 2
    SST = math.sqrt(varX * varY)

    return SSR / SST

# LOF

In [47]:
import pandas as pd
import numpy as np
data = pd.read_csv('Mathdataset.csv')

In [48]:
from sklearn.neighbors import LocalOutlierFactor
model = LocalOutlierFactor(n_neighbors=4, contamination=0.1,novelty=True)   
model.fit(data.iloc[:,:-1])

y = model._predict(data.iloc[:,:-1])
y

array([ 1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1, -1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

In [49]:
index = np.where(y == -1)[0]
index

array([ 7, 15])

In [50]:
for datum in range(len(index)):
    print('Outliers LOF : \n')
    print(data.iloc[index[datum],:],'\n')

Outliers LOF : 

X1     9.298388
X2    -8.881106
X3    -2.323673
X4     8.328420
Y     94.701501
Name: 7, dtype: float64 

Outliers LOF : 

X1     0.257540
X2    -8.067401
X3     4.662557
X4    -9.644255
Y    -10.769715
Name: 15, dtype: float64 



In [51]:
data_removed = data.drop(index,axis = 0)
data_removed

,X1,X2,X3,X4,Y
0,-9.956880,5.025726,2.863216,-8.696688,89.142195
1,2.919400,4.092015,-9.281015,0.099564,7.916545
2,-9.640004,-9.583399,-1.331211,-0.818754,93.020070
3,6.815343,1.667537,8.307681,-2.677693,44.646166
4,-6.032292,-7.632987,-5.934869,0.502384,36.083752
5,-3.080433,-7.665268,-6.800724,-8.814164,-0.160303
6,-9.403713,-8.959028,-4.658992,0.929124,89.124491
8,-2.495848,9.641412,5.272274,-9.512635,-3.687995
9,5.657371,-5.174477,9.563823,-0.674319,32.122093
10,-4.982524,6.592303,-4.423533,-2.863990,22.491840


In [52]:
print(eval_dataset(data))
print(eval_dataset(data_removed))

[0.51076217]
[0.55315656]


# iForest

In [53]:
from sklearn.ensemble import IsolationForest 
clf = IsolationForest( max_samples=100,random_state=0, contamination=0.05)
clf.fit(data.iloc[:,:-1])
y_pre = clf.predict(data.iloc[:,:-1])
y_pre

array([ 1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1])

In [54]:
index_2 = np.where(y_pre == -1)[0]
index_2

array([ 7, 23])

In [55]:
for datum in range(len(index_2)):
    print('Outliers LOF : \n')
    print(data.iloc[index_2[datum],:],'\n')

Outliers LOF : 

X1     9.298388
X2    -8.881106
X3    -2.323673
X4     8.328420
Y     94.701501
Name: 7, dtype: float64 

Outliers LOF : 

X1    -8.676233
X2     8.421807
X3     9.530712
X4    -9.936004
Y     66.079157
Name: 23, dtype: float64 



In [56]:
data_removed_2 = data.drop(index_2,axis = 0)
data_removed_2

,X1,X2,X3,X4,Y
0,-9.956880,5.025726,2.863216,-8.696688,89.142195
1,2.919400,4.092015,-9.281015,0.099564,7.916545
2,-9.640004,-9.583399,-1.331211,-0.818754,93.020070
3,6.815343,1.667537,8.307681,-2.677693,44.646166
4,-6.032292,-7.632987,-5.934869,0.502384,36.083752
5,-3.080433,-7.665268,-6.800724,-8.814164,-0.160303
6,-9.403713,-8.959028,-4.658992,0.929124,89.124491
8,-2.495848,9.641412,5.272274,-9.512635,-3.687995
9,5.657371,-5.174477,9.563823,-0.674319,32.122093
10,-4.982524,6.592303,-4.423533,-2.863990,22.491840


In [57]:
print(eval_dataset(data))
print(eval_dataset(data_removed_2))

[0.51076217]
[0.55853338]


# TCGPR

In [61]:
from TCGPR import TCGPR
dataSet = "Mathdataset.csv"
sampling_cap = 1
# for fdata screening (backward)
# ratio is recommend as a float <= 0.01
ratio = 0.1
up_search = 100

TCGPR.fit(filePath = dataSet,Sequence = 'backward',sampling_cap = sampling_cap, ratio = ratio,)

Modle Data screening of backward searching is executed : param initial_set_cap is masked 
The first execution of TCGPR : Data screening; backward version
It a greedy searching strategy, since the sampling_cap is 1
1-th iteration : The newly removed datum is
[ -8.64176059  -7.43055697  -9.08357688  -0.85692438 159.766893  ]
2-th iteration : The newly removed datum is
[ 9.29838752 -8.88110586 -2.32367347  8.32841968 94.70150104]
3-th iteration : The newly removed datum is
[-6.03229249 -7.63298697 -5.93486941  0.50238399 36.08375247]
****************************************************************************************************
The dataset after data screening is : 
          X1        X2        X3        X4           Y
0 -8.641761 -7.430557 -9.083577 -0.856924  159.766893
1  9.298388 -8.881106 -2.323673  8.328420   94.701501
2 -6.032292 -7.632987 -5.934869  0.502384   36.083752
The changes of fitting goodness is : 
 [array([[0.4110882]]), array([[0.60081511]]), array([[0.66589811]])